# **Especialização em Ciência de Dados - INF/UFRGS e SERPRO**
### Disciplina CD004 - Metodologia de Aprendizado de Máquina Supervisionado
#### *Profa. Mariana Recamonde-Mendoza (mrmendoza@inf.ufrgs.br)*
<br> 

---
***Observação:*** *Este notebook é disponibilizado aos alunos como complemento às aulas síncronas e aos slides preparados pela professora. Desta forma, os principais conceitos são apresentados no material teórico fornecido. O objetivo deste notebook é reforçar os conceitos e demonstrar questões práticas no uso de diferentes algoritmos e estratégias de Aprendizado de Máquina.*


---


<br>

## **Aula 01** - **Tópico: Avaliação de Modelos Preditivos**

<br>

Até o momento trabalhamos com a avaliação de modelos preditivos utilizando a estratégia mais simples, de **Holdout** - uma simples divisão aleatória dos dados em conjuntos de treinamento e teste (Holdout de 2 vias) ou em conjuntos de treinamento, validação e teste (Holdout de 3 vias). No entanto, um treinamento e avaliação robustos de modelos preditivos demanda o uso de outras estratégias de divisão de dados, especialmente quando estamos trabalhando com um conjunto de dados de tamanho limitado. Dentre estas estratégias, destacamos o *k-fold cross-validation*. 

O *k-fold cross-validation* (ou validação cruzada k-fold) e suas variantes (como *leave-one-out cross-validation* e *nested cross-validation*) estabeleceram-se como métodos referência para avaliar modelos em Aprendizado de Máquina pois mitigam diversas limitações relacionadas ao uso de Holdout, como i) permitir avaliar o desempenho de modelos preditivos com variações de conjuntos de dados de treinamento/teste (e assim evitar que os resultados dependam de uma escolha aleatória particular de divisão de dados), e ii) permitir que toda instância no conjunto de dados seja usada uma vez para avaliação do modelo.

<br> 

**Objetivo deste notebook**: Explorar o uso de k-fold cross-validation para treinamento e avaliação de modelos preditivos, bem como aplicar diferentes métricas de desempenho para avaliação de modelos de classificação.
<br>

---





##**Predição de risco de diabetes**

Os dados que utilizaremos neste notebook foram coletados no artigo de [Islam et al (2019)](https://link.springer.com/chapter/10.1007/978-981-13-8798-2_12) com o propósito de desenvolver um modelo para auxiliar no diagnóstico precoce de diabetes. O diagnóstico precoce só é possível através da avaliação adequada dos sintomas comuns e menos comuns, que podem ser observados em diferentes fases desde o início da doença até o diagnóstico. Os autores geraram um conjunto de dados com 520 instâncias, que foi coletado usando questionários diretos dos pacientes do Sylhet Diabetes Hospital em Sylhet, Bangladesh. Para realização desta atividade, não consideraremos o atributo idade (já descartado no conjunto de dados a ser baixado). Todos os atributos são binários, com respostas Sim/Não (Yes/No).



---



###Carregando e inspecionando os dados

Primeiramente, vamos carregar algumas bibliotecas importantes do Python e os dados a serem utilizados neste estudo. Os dados são disponibilizados através de um link, que também pode ser diretamente acessado pelos alunos.

In [ ]:
## Carregando as bibliotecas básicas necessárias
# A primeira linha é incluída para gerar os gráficos logo abaixo dos comandos de plot
%matplotlib inline              
import pandas as pd             # para análise de dados 
import matplotlib.pyplot as plt # para visualização de informações
import seaborn as sns           # para visualização de informações
import numpy as np              # para operações com arrays multidimensionais

## Bibliotecas para treinamento/avaliação de modelos
from sklearn.model_selection import RepeatedKFold, StratifiedKFold, train_test_split, cross_validate, cross_val_score, cross_val_predict
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier

## Bibliotecas para converter variáveis categóricas (strings) para numéricas
from sklearn.preprocessing import OrdinalEncoder 

sns.set()


In [ ]:
## Carregando os dados
df = pd.read_csv("https://drive.google.com/uc?export=view&id=1hSiFQPybmELwtIzbByASRxKp0TczCpn6")
df  

In [ ]:
## Características gerais do dataset
print("O conjunto de dados possui {} linhas e {} colunas".format(df.shape[0], df.shape[1]))

In [ ]:
## Avalia se existem dados duplicados
## A presença de dados duplicados pode 'inflar' nossa estimativa de desempenho
df.drop_duplicates(keep='last').shape
df = df.drop_duplicates(keep='last')

In [ ]:
## Características gerais do dataset - após remoção de duplicatas
print("O conjunto de dados possui {} linhas e {} colunas".format(df.shape[0], df.shape[1]))

A coluna *'DiabetesRisk'* contém a classificação de cada instância. Vamos avaliar a distribuição de classes do problema.

In [ ]:
## Distribuição do atributo alvo
plt.hist(df['DiabetesRisk'])
plt.title("Distribuição do atributo alvo")
plt.show()

In [ ]:
df['DiabetesRisk'].value_counts()

Podemos observar que neste conjunto de dados temos mais instâncias da classe 'DiabetesRisk' = 'Positive'. 

Também é importante averiguar (ou confirmar) os tipos de dados de cada coluna (atributo), bem como se existem valores faltantes (que serão identificados pelo Python com o comando `isnull()`, portanto, caso estejam codificados de outra forma, é necessário substituir por NaN).

In [ ]:
df.info()

In [ ]:
## Gerar um gráfico para cada variável categórica com a distribuição de 
## frequência entre as classes
def count_plot(df,columns,label):
    plt.figure(figsize=(16, 10))
    for indx, var  in enumerate(columns):
        plt.subplot(4, 4, indx+1)
        g = sns.countplot(x=var, data=df, hue=label)
    plt.tight_layout()

##Apenas as 15 primeiras colunas são atributos
count_plot(df,df.columns[:15],'DiabetesRisk') 


---


### Criando conjuntos de treino e teste para avaliação de modelos


Antes de iniciar o treinamento do modelo, lembre-se que é recomendado sempre reservar uma porção dos dados para teste, a qual somente será utilizada para avaliação do modelo final (após todo o processo de treinamento e otimização de hiperparâmetros).

Vamos fazer esta divisão, separando 20% para teste. Entretanto, primeiro precisamos dividir os dados entre atributos (X) e classe (y). Também iremos codificar os valores categóricos em inteiros a fim de ampliar as opções de algoritmos que podemos utilizar no treinamento dos modelos.



In [ ]:
## Separa o dataset em duas variáveis: os atributos/entradas (X) e a classe/saída (y)
X = df.drop(['DiabetesRisk'], axis=1)
y = df['DiabetesRisk'].values

In [ ]:
## Codifica variáveis categóricas usando inteiros. 
## Automaticamente detecta as categorias a partir dos dados. 
## Neste caso, codifica Yes/No -> 1/0
encoder = OrdinalEncoder(dtype=np.int64)
encoder.fit(X)
X = encoder.transform(X)

In [ ]:
## Imprime as 3 primeiras linhas após codificação, apenas para verificação
print(X[:3,:])

Faremos o mapeamento das classes Positive/Negative para 1/0. Por padrão, as funções de avaliação assumem que a classe 1 é a positiva/de interesse.

In [ ]:
## substitui 'Negative' por 0, 'Positive' por 1
y = np.array([0 if y=='Negative' else 1 for y in y]) 

In [ ]:
## Faz a divisão entre treino (80%) e teste (20%).
## O conjunto de treino representa os dados que serão usados
## ao longo do desenvolvimento do modelo

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20,stratify=y,random_state=42) 

---


### Treinamento de modelos com k-fold cross-validation

O scikit-learn possui amplo suporte para uso de k-fold cross-validation. Existem duas funções no scikit-learn que você pode usar para realizar a validação cruzada, a função [`cross_val_score`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html) e a função [`cross_validate`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html). A função `cross_val_score` está no scikit-learn há muito tempo e tem uma interface muito simples, enquanto a função `cross_validate` foi adicionada posteriormente, é um pouco mais poderosa e oferece mais opções (como especificar múltiplas métricas para avaliação). No entanto, ambas têm uma interface muito semelhante e são fáceis de usar. Uma questão importante é que elas permitem especificar tanto o algoritmo de aprendizado (estimator) como detalhes da divisão de dados (número de folds, [estratégia de divisão](https://scikit-learn.org/stable/modules/cross_validation.html#cross-validation-iterators), etc.) Por padrão, ambas usam 5-fold CV.

O K-fold cross-validation (K-fold CV) visa de certa forma 'substituir' uma simples divisão entre treino/validação para o desenvolvimento dos modelos. Assim, vamos aplicar o k-Fold CV na partição `X_train` usando uma árvore de decisão e um KNN (por enquanto, sem otimização de hiperparâmetros). Preparamos o procedimento de k-Fold CV antes da função a fim de gerar a mesma partição de treino/teste para cada fold no caso de comparar múltiplos algoritmos. Usamos a versão `StratifiedKFold` para gerar uma divisão estratificada em relação às classes (para regressão, ou para problemas bem balanceados, podemos usar `KFold`)

#### Usando a função cross_val_score

In [ ]:
cv_5f = StratifiedKFold(n_splits = 5, shuffle=True, random_state=42)

## Avalia uma árvore de decisão com 5-fold CV
clf_dt = DecisionTreeClassifier(max_depth=5,class_weight='balanced',random_state=42)
scores_dt = cross_val_score(estimator=clf_dt, X=X_train, y=y_train,scoring='f1',cv=cv_5f) 

## Avalia um 5-NN com 5-fold CV
clf_knn = KNeighborsClassifier(n_neighbors=5)
scores_knn = cross_val_score(estimator=clf_knn, X=X_train, y=y_train,scoring='f1',cv=cv_5f) 

In [ ]:
print(scores_dt)
print(np.mean(scores_dt))
print(np.std(scores_dt))

In [ ]:
print(scores_knn)
print(np.mean(scores_knn))
print(np.std(scores_knn))

In [ ]:
results=[]
results.append(scores_dt)
results.append(scores_knn)
plt.boxplot(results, labels=['DT','KNN'], showmeans=True)
plt.show()

A função [`cross_val_predict`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html) tem uma interface similar a `cross_val_score`, mas retorna a predição de cada instância quando a mesma foi alocada ao conjunto de teste.

In [ ]:
predict_dt = cross_val_predict(estimator=clf_dt, X=X_train, y=y_train,cv=cv_5f)#,method='predict_proba') ##descomente para retornar as probabilidades preditas
print(predict_dt)

#### Usando a função cross_validate

A função permite estabelecer um conjunto de métricas de avaliação no parâmetro [scoring](https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter). 

Vamos utilizar para fins de exemplo, F1, Recall, Average Precision (sumariza a curva precisão-recall como a média ponderada das precisões obtidas em cada threshold the probabilidade), área sob a curva ROC e Acurácia.

In [ ]:
scoring = ['f1', 'recall','average_precision','roc_auc','accuracy']

In [ ]:
## Avalia uma árvore de decisão com 5-fold CV
scores_dt2 = cross_validate(estimator=clf_dt, X=X_train, y=y_train,scoring=scoring,cv=cv_5f) 

## Avalia um 5-NN com 5-fold CV
scores_knn2 = cross_validate(estimator=clf_knn, X=X_train, y=y_train,scoring=scoring,cv=cv_5f) 

In [ ]:
scores_dt2_df = pd.DataFrame(scores_dt2, columns=scores_dt2.keys()) 
scores_dt2_df

In [ ]:
plt.boxplot(scores_dt2_df[["test_f1","test_recall","test_average_precision","test_roc_auc","test_accuracy"]].T,labels=['F1','Recall','AvgPrecision','ROC_AUC','Accuracy'],showmeans=True)
plt.show()

In [ ]:
scores_knn2_df = pd.DataFrame(scores_knn2, columns=scores_dt2.keys()) 
scores_knn2_df

In [ ]:
plt.boxplot(scores_knn2_df[["test_f1","test_recall","test_average_precision","test_roc_auc","test_accuracy"]].T,labels=['F1','Recall','AvgPrecision','ROC_AUC','Accuracy'],showmeans=True)
plt.show()

### Visualizando a divisão de dados com StratifiedKFold


Abaixo definimos uma funçao auxiliar para visualizar a divisão de dados feita pelo k-fold CV.

In [ ]:
#Fonte: https://scikit-learn.org/stable/auto_examples/model_selection/plot_cv_indices.html
cmap_data = plt.cm.Paired
cmap_cv = plt.cm.coolwarm

def plot_cv_indices(cv, X, y, ax, n_splits, lw=10):
    """Create a sample plot for indices of a cross-validation object."""

    # Generate the training/testing visualizations for each CV split
    for ii, (tr, tt) in enumerate(cv.split(X=X, y=y)):
        # Fill in indices with the training/test 
        indices = np.array([np.nan] * len(X))
        indices[tt] = 1
        indices[tr] = 0

        # Visualize the results
        ax.scatter(
            range(len(indices)),
            [ii + 0.5] * len(indices),
            c=indices,
            marker="_",
            lw=lw,
            cmap=cmap_cv,
            vmin=-0.2,
            vmax=1.2,
        )

    # Plot the data classes and groups at the end
    ax.scatter(
        range(len(X)), [ii + 1.5] * len(X), c=y, marker="_", lw=lw, cmap=cmap_data
    )

    # Formatting
    yticklabels = list(range(n_splits)) + ["class"]
    ax.set(
        yticks=np.arange(n_splits + 1) + 0.5,
        yticklabels=yticklabels,
        xlabel="Sample index",
        ylabel="CV iteration",
        ylim=[n_splits + 2.2, -0.2],
        xlim=[0, 100],
    )
    ax.set_title("{}".format(type(cv).__name__), fontsize=15)
    return ax

Na célula abaixo utilizamos a função `plot_cv_indices` para visualizar a distribuição das instâncias entre folds de treino e teste no 5-fold CV definido anteriormente.

In [ ]:
from matplotlib.patches import Patch

fig, ax = plt.subplots()
plot_cv_indices(cv_5f, X_train, y_train, ax, n_splits=5)
ax.legend(
        [Patch(color=cmap_cv(0.8)), Patch(color=cmap_cv(0.02))],
        ["Testing set", "Training set"],
        loc=(1.02, 0.8),
    )

---

### Treinamento de modelos com repeated k-fold cross-validation

Podemos repetir o processo de k-fold CV múltiplas vezes utilizando a função `RepeatedKFold`. È possível perceber no gráfico abaixo que a cada repetição da divisão das instâncias em folds, obtemos uma "configuração" diferente. Na prática, passamos a ter a avaliação do modelo com 15 conjuntos de teste bem distintos. 

Entretanto, é importante perceber que separar os dados em 5 folds e repetir o processo 3 vezes é **diferente** de separar os dados em 15 folds e repetir o processo uma vez. No geral, dividir em menos folds e repetir o processo tantas vezes quantas forem possíveis é mais indicado, por permitir uma avaliação com uma maior variedade na distribuição dos dados.

In [ ]:
repcv_5f = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)


In [ ]:
fig, ax = plt.subplots()
plot_cv_indices(repcv_5f, X_train, y_train, ax, n_splits=15)
ax.legend(
        [Patch(color=cmap_cv(0.8)), Patch(color=cmap_cv(0.02))],
        ["Testing set", "Training set"],
        loc=(1.02, 0.8),
    )

In [ ]:
cv_15f = StratifiedKFold(n_splits=15, shuffle=True,random_state=42)

fig, ax = plt.subplots()
plot_cv_indices(cv_15f, X_train, y_train, ax, n_splits=15)
ax.legend(
        [Patch(color=cmap_cv(0.8)), Patch(color=cmap_cv(0.02))],
        ["Testing set", "Training set"],
        loc=(1.02, 0.8),
    )

Abaixo vamos realizar o treinamento dos dois modelos, baseado em árvores de decisão e baseado em KNN, utilizando o RepeatedKFold. O processo é exatamente igual ao que foi feito anteriormente, a diferença é que agora obtemos 15 estimativas de desempenho para cada modelo.

In [ ]:
## Avalia uma árvore de decisão com 5-fold CV repetido 3 vezes
scores_dt_rep = cross_validate(estimator=clf_dt, X=X_train, y=y_train,scoring=scoring,cv=repcv_5f) 

## Avalia um 5-NN  com 5-fold CV repetido 3 vezes
scores_knn_rep = cross_validate(estimator=clf_knn, X=X_train, y=y_train,scoring=scoring,cv=repcv_5f) 

In [ ]:
scores_dt2rep_df = pd.DataFrame(scores_dt_rep, columns=scores_dt_rep.keys()) 
scores_dt2rep_df

In [ ]:
plt.boxplot(scores_dt2rep_df[["test_f1","test_recall","test_average_precision","test_roc_auc","test_accuracy"]].T,labels=['F1','Recall','AvgPrecision','ROC_AUC','Accuracy'],showmeans=True)
plt.show()

In [ ]:
scores_knn2rep_df = pd.DataFrame(scores_knn_rep, columns=scores_knn_rep.keys()) 
scores_knn2rep_df

In [ ]:
plt.boxplot(scores_knn2rep_df[["test_f1","test_recall","test_average_precision","test_roc_auc","test_accuracy"]].T,labels=['F1','Recall','AvgPrecision','ROC_AUC','Accuracy'],showmeans=True)
plt.show()

Para avaliar o quanto os nossos modelos parecem promissores perto de um classificador 'dummy', sem nenhuma inteligência mas sim baseado em alguns critérios pré-definidos (como a classe majoritária), podemos utilizar a função `DummyClassifier`. Ela permite gerar um modelo 'baseline', que ignora os dados de entrada (atributos) e apenas olha a distribuição dos valores de saída nos dados de treinamento. A estratégia 'most_frequent' retorna sempre a classe mais frequente.

In [ ]:
from sklearn.dummy import DummyClassifier
clf_dummy = DummyClassifier(strategy='most_frequent', random_state=42)
scores_dummy_rep = cross_validate(estimator=clf_dummy, X=X_train, y=y_train,scoring=scoring,cv=repcv_5f) 

In [ ]:
scores_dummyrep_df = pd.DataFrame(scores_dummy_rep, columns=scores_dummy_rep.keys()) 
scores_dummyrep_df

In [ ]:
print(np.mean(scores_dt2rep_df))
print(np.mean(scores_knn2rep_df))
print(np.mean(scores_dummyrep_df))

Pelo critério de F1-Score, o modelo de KNN (com k = 5) obteve os melhores resultados. Assim, este algoritmo poderia ser escolhido dentre os dois analisados (árvore de decisão e KNN) para treinar um modelo a partir de todos os dados de treinamento, e avaliar seu desempenho final nos dados de teste. Este desempenho final é a estimativa do poder preditivo que poderíamos obter caso este modelo seja aplicado para auxiliar no diagnóstico precoce de diabetes. 

In [ ]:
#from sklearn.metrics import roc_curve, auc
knn_k5 = clf_knn.fit(X_train,y_train)
y_predProba_knn = knn_k5.predict_proba(X_test)
y_pred_knn = knn_k5.predict(X_test)

print(metrics.accuracy_score(y_test,y_pred_knn))
print(metrics.f1_score(y_test,y_pred_knn))

In [ ]:
metrics.RocCurveDisplay.from_estimator(knn_k5, X_test, y_test)

# alternativamente, a curva ROC pode ser gerada a partir das predições:
# fpr, tpr, thresholds = roc_curve(y_test,y_predProba_dt[:,1], pos_label=1)
# roc_auc = metrics.auc(fpr, tpr)
# display = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=roc_auc,estimator_name='example estimator')
# display.plot()

Visualizando a curva de Precisão-Recall:

In [ ]:
## Análise da curva PR
metrics.PrecisionRecallDisplay.from_estimator(knn_k5, X_test, y_test)

---

### Otimização de hiperparâmetros com GridSearch e Nested Cross-validation

O sklearn disponibiliza a função `GridSearchCV`, que permite realizar a otimização de hiperparâmetros de forma prática usando k-fold cross-validation. Esta função já foi utilizada em exercícios anteriores da disciplina CD003, entretanto, na ocasião configuramos a função para utilizar splits de treino/validação pré-definidos. A chamada da função é muito simples:



```
search = GridSearchCV(estimator, param_grid, scoring='f1', n_jobs=1, cv=5, refit=True)
```

A função utiliza um modelo (`estimator`) para explorar as combinações de valores de hiperparâmetros definidos em `param_grid,` através de um 5-fold cross-validation (determinado por `cv`) e escolhe o melhor modelo a partir da métrica de F1-Score (também configurável). A opção `refit=True` define que após determinar a melhor configuração de hiperparâmetros, a mesma será utilizada para retreinar um modelo com todos os dados utilizados no processo de GridSearch com CV.



A validação cruzada aninhada (nested cross-validation) é frequentemente usada para treinar um modelo no qual os hiperparâmetros também precisam ser otimizados. A seleção de modelo sem CV aninhado usa os mesmos dados para ajustar os hiperparâmetros do modelo e avaliar o desempenho do modelo. As informações podem, portanto, “vazar” para o modelo, que acaba se sobreajustando aos dados. O resultado deste vazamento pode ser uma avaliação excessivamente otimista. A magnitude desse efeito depende principalmente do tamanho do conjunto de dados e da estabilidade do modelo. 

Na célula abaixo, vamos executar o k-fold cross-validation e o nested cross-validation para os dados de risco de diabetes, treinando uma árvore de decisão com pré-pode. O hiperparâmetro a ser otimizado é o `max_depth`. Para os resultados mostrados abaixo, o desempenho refere-se sempre ao melhor valor de hiperparâmetro obtido a cada iteração do GridSearch.

In [ ]:
#Adaptado de: https://scikit-learn.org/stable/auto_examples/model_selection/plot_nested_cross_validation_iris.html

from sklearn.model_selection import GridSearchCV

# Número de execuções da validação cruzada (aninhada/não aninhada)
NUM_TRIALS = 30

# Grid de hiperparâmetros para otimização
# param_grid = {"max_depth": [1,2,3,4,5,6,7]}
param_grid = {"n_neighbors": [1,3,5,7,9]}

# Uso de uma árvore de decisão como algoritmo de aprendizado
# model = DecisionTreeClassifier()
model = KNeighborsClassifier()

# Arrays para armazenar os scores de cada abordagem
non_nested_scores = np.zeros(NUM_TRIALS)
nested_scores = np.zeros(NUM_TRIALS)

# Loop para múltiplas execuções, cada qual com um random_state distinto
for i in range(NUM_TRIALS):

    # Definir estratégias de validação cruzada para o loop interno (inner) e para
    # o loop externo (outer). As escolhas podem ser diferentes. Aqui, optamos por
    # usar StratifiedKFold em ambas.
    inner_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=i)
    outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=i)

    # GridSearch e avaliação dos modelos na abordagem de k-fold cross-validation padrão
    # (sem aninhamento) - avalia os modelos com os mesmos dados usados para selecionar hiperparâmetros
    clf = GridSearchCV(estimator=model, param_grid=param_grid, cv=outer_cv) ## sem aninhamento - desempenho com melhores hiperparâmetros
    clf.fit(X_train, y_train)
    non_nested_scores[i] = clf.best_score_

    # GridSearch e avaliação dos modelos na abordagem de nested k-fold cross-validation
    clf = GridSearchCV(estimator=model, param_grid=param_grid, cv=inner_cv, refit=True) ##inner CV - melhores hiperparâmetros
    nested_score = cross_val_score(clf, X=X_train, y=y_train, cv=outer_cv) ##outer CV - desempenho com melhores hiperparâmetros (do inner)
    nested_scores[i] = nested_score.mean()

## Calcula a diferença de scores entre as abordagens
score_difference = non_nested_scores - nested_scores

print(
    "Average difference of {:6f} with std. dev. of {:6f}.".format(
        score_difference.mean(), score_difference.std()
    )
)

# Plotar um gráfico com os scores obtidos e com a diferença entre eles
# em cada iteração
plt.figure(figsize=(15,15))
plt.subplot(211)
(non_nested_scores_line,) = plt.plot(non_nested_scores, color="r")
(nested_line,) = plt.plot(nested_scores, color="b")
plt.ylabel("score", fontsize="14")
plt.legend(
    [non_nested_scores_line, nested_line],
    ["Non-Nested CV", "Nested CV"],
    bbox_to_anchor=(0, 0.4, 0.5, 0),
)
plt.title(
    "Non-Nested and Nested Cross Validation on Diabetes Dataset",
    x=0.5,
    y=1.1,
    fontsize="15",
)

# Plot bar chart of the difference.
plt.subplot(212)
difference_plot = plt.bar(range(NUM_TRIALS), score_difference)
plt.xlabel("Individual Trial #")
plt.legend(
    [difference_plot],
    ["Non-Nested CV - Nested CV Score"],
    bbox_to_anchor=(0, 1, 0.8, 0),
)
plt.ylabel("score difference", fontsize="14")

plt.show()

A célula a seguir faz a otimização de hiperparâmetros com nested cross-validation para o algoritmo KNN. O mesmo processo poderia ser adaptado ao algoritmo Árvores de Decisão ou outros que envolvam a busca por melhores hiperparâmetros.

In [ ]:
# define o modelo
model = KNeighborsClassifier()
# define o espaço de busca de hiperparâmetros
param_grid = dict()
param_grid['n_neighbors'] = [1,3,5,7,9]

### loop interno ####
# configura o loop interno do nested cross-validation
cv_inner = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

# define a estratégia de busca dos melhores hiperparâmetros (baseado em F1)
search = GridSearchCV(model, param_grid, scoring='f1', n_jobs=1, cv=cv_inner, refit=True)
### loop interno ####

### loop externo ####
# configura o loop externo do nested cross-validatiion
cv_outer = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)

# executa o nested cross-validation
output_ncv = cross_validate(search, X_train, y_train, scoring=scoring, cv=cv_outer, n_jobs=-1,return_estimator=True, return_train_score=True)
### loop externo ####

# reporta os resultados
pd.DataFrame(output_ncv)

Podemos avaliar os resultados do processo de nested cross-validation observando a variação de desempenho ao longo das múltiplas execuções e os melhores valores de hiperparâmetros selecionados em cada iteração. Cabe salientar que esta análise é feita com base no número de repetições do outer cross-validation (neste caso, configurado com 10 folds)

In [ ]:
# Análise de f1
mean_val_score = output_ncv['test_f1'].mean()

print('nested_train_scores: ', output_ncv['train_f1'])
print('nested_val_scores:   ', output_ncv['test_f1'])
print('mean score:            {0:.2f}'.format(mean_val_score))

In [ ]:
# Análise de ROC AUC
mean_val_score = output_ncv['test_roc_auc'].mean()

print('nested_train_scores: ', output_ncv['train_roc_auc'])
print('nested_val_scores:   ', output_ncv['test_roc_auc'])
print('mean score:            {0:.2f}'.format(mean_val_score))

In [ ]:
## Quais os melhores valores de hiperparâmetros de acordo com o nested Cross_validation?
[x.best_params_ for x in output_ncv['estimator']]

O conhecimento a respeito dos melhores hiperparâmetros ao longo do nested cross-validation pode guiar nossa decisão sobre que modelos (algoritmos e configuração de hiperparâmetros) podemos utilizar para gerar o modelo final. Este modelo final seria obtido treinando um modelo com estas configurações sobre todo o conjunto de dados usado no processo de avaliação. 

A célula abaixo exemplifica este processo:

In [ ]:
## Treina o modelo com o valor "ótimo" do hiperparâmetro k
clf_knnOptimal = KNeighborsClassifier(n_neighbors=9)
clf_knnOptimal.fit(X_train, y_train)

## Apĺica o modelo treinado para prever a saída dos dados de teste retidos no início da atividade
y_pred_knnOptimal = clf_knnOptimal.predict(X_test)

## Análise da curva ROC
metrics.RocCurveDisplay.from_estimator(clf_knnOptimal, X_test, y_test)

## Análise da curva PR
metrics.PrecisionRecallDisplay.from_estimator(clf_knnOptimal, X_test, y_test)


In [ ]:
print(metrics.f1_score(y_test,y_pred_knnOptimal))